## Use the gene names to parse protein groups tables for the protein identifier. Take the matching gene and corresponding protein name to search for protein sequence

Number of isoforms detected in AM protein groups 2 <br>
Tmpo <br>
Tor1aip2 <br>

Number of isoforms detected in AM protein groups 1<br>
Tmpo <br>



In [423]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from collections import Counter


In [101]:
proteomics_AM_GeneNames = pd.read_csv('../../Data/RnaProtein_dropbox/PPG_AM_proteomics_covariance_input_0419.tsv',sep='\t',usecols=[0])
proteomics_AM_GeneNames.set_index('Gene.names',inplace=True)
proteomics_AT_GeneNames = pd.read_csv('../../Data/RnaProtein_dropbox/PPG_AT2_proteomics_covariance_input_0419.tsv',sep='\t',usecols=[0])
proteomics_AT_GeneNames.set_index('Gene.names',inplace=True)

In [494]:
proteomics_AM_GeneNames.shape

(4017, 0)

In [495]:
am_mappings.shape

(4017, 3)

In [67]:
GeneNames = list(proteomics_AM_GeneNames.index) + list(proteomics_AT_GeneNames.index)
GeneNames = set(GeneNames) #list of all unique gene names found in both AM and AT2 proteomics tables

In [298]:
# ProtGrps_AM = pd.read_csv('../../Data/PPG_ProteinGroups/proteinGroups_AM.txt',sep='\t',usecols=[0,1,2,5,6])
# ProtGrps_AM_missing = ProtGrps_AM[ProtGrps_AM['Gene names'].isna()]
# ProtGrps_AM.drop(labels=ProtGrps_AM[ProtGrps_AM['Gene names'].isna()].index,inplace=True)
# ProtGrps_AM.set_index('Gene names',inplace=True)

# ProtGrps_AT2 = pd.read_csv('../../Data/PPG_ProteinGroups/proteinGroups_AT.txt',sep='\t',usecols=[0,1,2,5,6])
# ProtGrps_AT2_missing = ProtGrps_AT2[ProtGrps_AT2['Gene names'].isna()]
# ProtGrps_AT2.drop(labels=ProtGrps_AT2[ProtGrps_AT2['Gene names'].isna()].index,inplace=True)
# ProtGrps_AT2.set_index('Gene names',inplace=True)

In [340]:
AM_imputed = pd.read_csv('../../Data/PPG_ProteinGroups/PPG_AM_main_imputed.tsv',sep='\t',usecols=[0,1,2])
AM_imputed.set_index('Gene.names',inplace=True)
AT2_imputed = pd.read_csv('../../Data/PPG_ProteinGroups/PPG_AT2_main_imputed.tsv',sep='\t',usecols=[0,1,2])
AT2_imputed.set_index('Gene.names',inplace=True)

### For the gene names found in the proteomics table extract protein names from imputed, check for differences in majority protein

In [376]:
am_genes = list(set(list(AM_imputed.index))&GeneNames) #overlapping gene names from inputed and protein covariance
at2_genes = list(set(list(AT2_imputed.index))&GeneNames) #overlapping gene names from inputed and protein covariance

In [377]:
am_mappings = AM_imputed.loc[am_genes,:].copy()
am_mappings['Majority.protein.IDs'] = am_mappings['Majority.protein.IDs'].str.split(';').str[0]

at2_mappings = AT2_imputed.loc[at2_genes,:].copy()
at2_mappings['Majority.protein.IDs'] = at2_mappings['Majority.protein.IDs'].str.split(';').str[0]

### shows if there is a different majority protein in the first position between am and at2 cells

In [439]:
for gene in set(am_mappings.index)&set(at2_mappings.index):
    if am_mappings.loc[gene,'Majority.protein.IDs'] != at2_mappings.loc[gene,'Majority.protein.IDs']:
        print(gene)
        print(am_mappings.loc[gene,'Majority.protein.IDs'])
        print(at2_mappings.loc[gene,'Majority.protein.IDs'])
        print('\n')

Igkv3-2
P01654
P03977




#### List below to grab fasta files from uniprot

In [398]:
uniprot_names = list(set(am_mappings['Majority.protein.IDs']) | set(at2_mappings['Majority.protein.IDs']))

In [405]:
with open("Data/uniprot.txt","w+")  as fp:
    for name in uniprot_names:
        fp.write(name+'\n')

### combine fasta file to corresponding protein names

In [415]:
with open("Data/protein_seq.fasta","r+") as fp:
    sequences = fp.read().splitlines()

In [463]:
counter = 0
sequence_mappings = {protein: [] for protein in uniprot_names} #initialize dict, protein names as keys and value is a list

for line in sequences[1::]: #skip first line
    if '>' in line:
        counter += 1
        continue #immediately goes to next iteration
    else:
        sequence_mappings[uniprot_names[counter]].append(line)
        
#concatenate all lists into strings
for keys in sequence_mappings.keys():
    sequence_mappings[keys] = ''.join(sequence_mappings[keys])

In [471]:
#Use sequence mappings to match corresponding sequence to protein
at2_mappings['ProteinSequence'] = at2_mappings['Majority.protein.IDs'].map(sequence_mappings)
am_mappings['ProteinSequence'] = am_mappings['Majority.protein.IDs'].map(sequence_mappings)

In [493]:
at2_mappings.to_csv('Data/AT2_SeqMappings.tsv',sep='\t')
am_mappings.to_csv('Data/AM_SeqMappings.tsv',sep='\t')